In [1]:
# See : https://github.com/h2oai/h2o-training-book/blob/master/hands-on_training/anomaly_detection.md
## h2o-training-book/package.json :: "license": "Apache 2",

# Anomaly Detection on MNIST

This notebook shows how a Deep Learning Auto-Encoder model can be used to find outliers in a dataset. 

Consider the following three-layer neural network with one hidden layer and the same number of input neurons (features) as output neurons.  The loss function is the MSE between the input and the output.  Hence, the network is forced to learn the identity via a nonlinear, reduced representation of the original data.  Such an algorithm is called a deep autoencoder; these models have been used extensively for unsupervised, layer-wise pretraining of supervised deep learning tasks, but here we consider the autoencoder's application for discovering anomalies in data.

We use the well-known MNIST dataset of hand-written digits, where each row contains the 28^2=784 raw gray-scale pixel values from 0 to 255 of the digitized digits (0 to 9).

## Load Theano/Lasagne and the MNIST training/testing datasets

In [ ]:
We do unsupervised training, so we can drop the response column.

train_hex <- train_hex[,-resp]
test_hex <- test_hex[,-resp]

## Finding outliers - ugly hand-written digits

We train a Deep Learning Auto-Encoder to learn a compressed (low-dimensional) non-linear representation of the dataset, hence learning the intrinsic structure of the training dataset. The auto-encoder model is then used to transform all test set images to their reconstructed images, by passing through the lower-dimensional neural network. We then find outliers in a test dataset by comparing the reconstruction of each scanned digit with its original pixel values. The idea is that a high reconstruction error of a digit indicates that the test set point doesn't conform to the structure of the training data and can hence be called an outlier.

### Learn what's normal from the training data

Train unsupervised Deep Learning autoencoder model on the training dataset. For simplicity, we train a model with 1 hidden layer of 50 Tanh neurons to create 50 non-linear features with which to reconstruct the original dataset. We learned from the Dimensionality Reduction tutorial that 50 is a reasonable choice. For simplicity, we train the auto-encoder for only 1 epoch (one pass over the data). We explicitly include constant columns (all white background) for the visualization to be easier.

### Find outliers in the test data

The Anomaly app computes the per-row reconstruction error for the test data set. It passes it through the autoencoder model (built on the training data) and computes mean square error (MSE) for each row in the test set.

In [ ]:
test_rec_error <- as.data.frame(h2o.anomaly(test_hex, ae_model))

In case you wanted to see the lower-dimensional features created by the auto-encoder deep learning model, here's a way to extract them for a given dataset. This a non-linear dimensionality reduction, similar to PCA, but the values are capped by the activation function (in this case, they range from -1...1)

In [ ]:
test_features_deep <- h2o.deepfeatures(test_hex, ae_model, layer=1)
summary(test_features_deep)

## Visualize the good, the bad and the *ugly*

We will need a helper function for plotting handwritten digits (adapted from http://www.r-bloggers.com/the-essence-of-a-handwritten-digit/). Don't worry if you don't follow this code...

Let's look at the test set points with low/median/high reconstruction errors. We will now visualize the original test set points and their reconstructions obtained by propagating them through the narrow neural net.

### The good

Let's plot the 25 digits with lowest reconstruction error. First we plot the reconstruction, then the original scanned images.

Clearly, a well-written digit 1 appears in both the training and testing set, and is easy to reconstruct by the autoencoder with minimal reconstruction error. Nothing is as easy as a straight line.

### The bad

Now let's look at the 25 digits with median reconstruction error.

These test set digits look "normal" - it is plausible that they resemble digits from the training data to a large extent, but they do have some particularities that cause some reconstruction error.

### The ugly

And here are the biggest outliers - The 25 digits with highest reconstruction error!

Now here are some pretty ugly digits that are plausibly not commonly found in the training data - some are even hard to classify by humans.

## Voila!

We were able to find outliers with Deep Learning Auto-Encoder models.  We would love to hear your usecase for Anomaly detection.
